
# **Random Search Algorithm**

The random search algorithm works similarly, but instead of using a predetermined list of hyperparameter values, the values are randomly chosen. As with grid search, it selects the hyperparameter that performed the best.


![image](images/random_search.png)

To implement this, one needs to search over hyperparameters by drawing a list of random values from distributions. What would this mean for say, the hyperparameter `C` in logistic regression that we’ve been working with so far? Instead of selecting from a pre-determined list (like `[1,10,100]` which we’ve been using for grid search in previous exercises), we would draw 3 random values between say 0 and 100.

In scikit-learn, the `RandomSearchCV` function implements random search with cross-validation. `RandomSearchCV` requires three arguments to be specified:

- `estimator`: the machine learning model whose hyperparameters we’re tuning; this is exactly the same for GridSearchCV
- `param_distributions`: a dictionary which specifies the hyperparameters as keys and corresponding distributions to draw lists of values from for each hyperparameter. In GridSearchCV, we instead had param_grid, a dictionary representing the grid of hyperparameters to search from
- `n_iter`: the number of times the algorithm needs to randomly draw from the distributions. The default value for this is 10.

With random search, we don’t have to make a list, but we still have to provide some information about how we want to select random numbers. Do we want random numbers between 0 and 100? Between -10 and 10? Do we want the same chance of picking small numbers and picking large numbers?

We can do this by specifying a probability distribution for each hyperparameter.

In [ ]:
from scipy.stats import uniform

distributions = {"penalty": ["l1", "l2"], "C": uniform(loc=0, scale=100)}

- The `penalty` hyperparameter of scikit-learn’s `LogisticRegression` model has only two possible values: l1 and l2. We list them both. `RandomizedSearchCV` will treat this as a discrete uniform distribution. This just means that every item in the list has an equal chance of being selected. In this case, there’s a 50% chance of drawing l1 and a 50% chance of drawing l2.
- The hyperparameter `C` is the inverse of regularization strength. It can be any positive number, so we have to specify a probability distribution that allows us to randomly select a positive number. The `scipy` library has many probability distributions to choose from. For this example, we’re using the uniform distribution. This allows us to randomly select numbers between `loc` and `loc + scale` (in this case, between 0 and 100).

In [ ]:
first_draw = distributions["C"].rvs(10)
second_draw = distributions["C"].rvs(10)
print(first_draw)
print(second_draw)

[98.17561301 15.68609911 66.25615582 95.87047921 15.12271514 20.24596147
 56.92515023 82.20232609 63.76772312 89.40235078]
[95.04578112 33.89459268 77.55846237  2.17034458 39.35571902  7.83673503
 17.36575256 86.49431442 10.55323588 22.93857171]


## **sklearn.model_selection.RandomizedSearchCV**

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split

# Load the data set
cancer = load_breast_cancer()

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target)

lr = LogisticRegression(solver="liblinear", max_iter=1000)

# Create a RandomizedSearchCV model
clf = RandomizedSearchCV(lr, distributions, n_iter=8)

After fitting a `RandomizedSearchCV` model we can find out the results using the following attributes of the `clf` argument:

- `.best_estimator_` gives us the best estimator
- `.best_score_` gives us the mean cross-validated score corresponding to the best estimator
- `.best_params_` gives us the set of hyperparameters that correspond to the best estimator

In [ ]:
clf.fit(X_train, y_train)
best_model = clf.best_estimator_
print(best_model)
print(clf.best_params_)

LogisticRegression(C=np.float64(62.90838954241394), max_iter=1000, penalty='l1',
                   solver='liblinear')
{'C': np.float64(62.90838954241394), 'penalty': 'l1'}


In [ ]:
best_score = clf.best_score_
test_score = clf.score(X_test, y_test)
print(best_score)
print(test_score)

0.9554856361149111
0.9790209790209791


In [ ]:
import pandas as pd

hyperparameter_values = pd.DataFrame(clf.cv_results_["params"])
randomsearch_scores = pd.DataFrame(clf.cv_results_["mean_test_score"], columns=["score"])

df = pd.concat([hyperparameter_values, randomsearch_scores], axis=1)
print(df)

           C penalty     score
0  52.263196      l2  0.950780
1  62.908390      l1  0.955486
2  61.419905      l1  0.955486
3  97.818871      l1  0.950807
4  83.886440      l1  0.955458
5  77.980712      l2  0.948427
6  54.864810      l1  0.955486
7  53.429513      l2  0.948427
